<a href="https://colab.research.google.com/github/sn1kyy/Weather-forecasting-/blob/main/weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1. import libraries

In [11]:
import requests #helps in fetching data from APIs
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [12]:
#the API key information
api_key = '46521bf9fb4a8ee8b1672f3e7eb8f9a4'
BASE_URL = 'https://api.openweathermap.org/data/2.5/weather?lat=44.34&lon=10.99&appid=46521bf9fb4a8ee8b1672f3e7eb8f9a4'

2. Fetch current weather data

In [13]:
#getting the information on the entered city
def get_current_data(city):
  api_key = '46521bf9fb4a8ee8b1672f3e7eb8f9a4'
  url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
  response = requests.get(url) #sends the get request to API(js like we did previously)
  data = response.json()
  print(data)
  if 'name' not in data or 'main' not in data:
        print(f"Error: Could not find weather data for '{city}'. Response: {data}")
        return None
  return{
      'city' : data['name'],
      'current_temp' : round(data['main']['temp']),
      'feels_like' : round(data['main']['feels_like']),
      'temp_min' : round(data['main']['temp_min']),
      'temp_max' : round(data['main']['temp_max']),
      'humidity' : round(data['main']['humidity']),
      #round is for numerical data only
      'description' : data['weather'][0]['description'],
      'country' : data['sys']['country'],
      'Wind_Gust_Speed' : data['wind']['speed'],
      'WindGustDir' : data['wind']['deg'],
      'pressure': data['main']['pressure']
  }

3. Read historical data

In [14]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

4. Preparing data - (a)Converting categorial data into numerical values

In [15]:
def prepare_data(data):
  le = LabelEncoder() #creates the labelencoder
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])


  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
  y = data['RainTomorrow']

  return X, y, le

(b)Training the model

In [16]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)
  return model

  y_pred = model.predict(X_test)
  accuracy = mean_squared_error(y_test, y_pred)
  print("Accuracy: {accuracy}")

  return model

(c)Preparing regression data

In [17]:
def prepare_regression_data(data, feature):
  X,y = [], []

  for i in range(len(data)-1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1,1)
  y = np.array(y)
  return X, y


(d)Training regression data

In [18]:
def train_regression_model(X, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y)
  return model

(e)Predicting

In [19]:
def predict(model, current_value):
  prediction = [current_value]

  for i in range(5):
    next_value = model.predict(np.array(prediction).reshape(-1,1))
    prediction.append(next_value[0])

  return prediction[1:]

5.**Weather anlysis function**

In [21]:
def weather_view():
  #getting the city name
  city = input("Enter the city name: ")
  current_weather = get_current_data(city)

  if current_weather is None:
        print("Please enter a valid city name.")
        return

  #loading historical data
  historical_data = read_historical_data('/content/weather.csv')

  #preparing and training the model
  X, y, le = prepare_data(historical_data)
  rain_model = train_rain_model(X, y)


  #map wind direction to compass points
  wind_deg = current_weather['WindGustDir'] % 360
  compass_points = [
        ("Ν", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ΕΝΕ", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
        ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1


  #fetching current data
  current_data ={
      'MinTemp' : current_weather['temp_min'],
      'MaxTemp' : current_weather['temp_max'],
      'WindGustDir' : compass_direction_encoded,
      'WindGustSpeed' : current_weather['Wind_Gust_Speed'],
      'Humidity' : current_weather['humidity'],
      'Pressure' : current_weather['pressure'],
      'Temp' : current_weather['current_temp']
  }

  current_df = pd.DataFrame([current_data])

  #predicting rain
  rain_prediction = rain_model.predict(current_df)[0]

  #predicting future temp and humidity
  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, y_temp)
  hum_model = train_regression_model(X_hum, y_hum)

  temp_prediction = predict(temp_model, current_weather['temp_min'])
  hum_prediction = predict(hum_model, current_weather['humidity'])


  timezone = pytz.timezone('Asia/Kolkata')
  now = datetime.now(timezone)
  next_hours = now + timedelta(hours=1)
  next_hours = next_hours.replace(minute=0, second=0, microsecond=0)

  future_times = [next_hours + timedelta(hours=i) for i in range(5)]

  print(f"City: {city}, {current_weather['country']}") # Fixed syntax error
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Wind Gust Speed: {current_weather['Wind_Gust_Speed']} m/s")
  print(f"Minimum Tempereture:{current_weather['temp_min']}°C")
  print(f"Maximun Tempereture:{current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Description: {current_weather['description']}")
  print(f"Rain Prediction:{' Yes' if rain_prediction else ' No' } ") # Corrected rain prediction variable


weather_view()

Enter the city name: delhi
{'coord': {'lon': 77.2167, 'lat': 28.6667}, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50d'}], 'base': 'stations', 'main': {'temp': 29.05, 'feels_like': 36.05, 'temp_min': 29.05, 'temp_max': 29.05, 'pressure': 1000, 'humidity': 89, 'sea_level': 1000, 'grnd_level': 975}, 'visibility': 3000, 'wind': {'speed': 1.03, 'deg': 0}, 'clouds': {'all': 100}, 'dt': 1752225246, 'sys': {'type': 1, 'id': 9165, 'country': 'IN', 'sunrise': 1752192052, 'sunset': 1752241916}, 'timezone': 19800, 'id': 1273294, 'name': 'Delhi', 'cod': 200}
City: delhi, IN
Current Temperature: 29°C
Feels Like: 36°C
Wind Gust Speed: 1.03 m/s
Minimum Tempereture:29°C
Maximun Tempereture:29°C
Humidity: 89%
Description: mist
Rain Prediction: Yes 
